In [1]:
import pandas as pd
import os
import numpy as np

Importing and combining flight data sources and aircraft data source

In [2]:
#appending the monthly flight data sources into a single dataframe(takes approx 3-4 min to run) 
flights_db = pd.read_csv("Monthly Data Sources/Flight/1-Jan 2023.csv")


In [3]:
aircrafts_lookup = pd.read_csv("Monthly Data Sources/Aircraft/Aircraft.csv")

C:\Users\jenso\AppData\Local\Temp\ipykernel_16256\1809701077.py:1: DtypeWarning: Columns (18) have mixed types. Specify dtype option on import or set low_memory=False.
  aircrafts_lookup = pd.read_csv("Monthly Data Sources/Aircraft/Aircraft.csv")


In [4]:
#creating lookup table source for the flight dataset
airline_look_up = pd.read_csv("Look Up Tables/L_AIRLINE_ID.csv")
airport_look_up = pd.read_csv("Look Up Tables/L_AIRPORT_ID.csv")
cancellation_look_up = pd.read_csv("Look Up Tables/L_CANCELLATION.csv")
country_look_up = pd.read_csv("Look Up Tables/L_WORLD_AREA_CODES.csv")

Merging the dataset together

In [5]:
flights_db.dtypes

YEAR                       int64
MONTH                      int64
DAY_OF_MONTH               int64
FL_DATE                   object
OP_CARRIER_AIRLINE_ID      int64
TAIL_NUM                  object
OP_CARRIER_FL_NUM          int64
ORIGIN_AIRPORT_ID          int64
ORIGIN_AIRPORT_SEQ_ID      int64
ORIGIN_CITY_MARKET_ID      int64
ORIGIN                    object
ORIGIN_CITY_NAME          object
ORIGIN_WAC                 int64
DEST_AIRPORT_ID            int64
DEST_AIRPORT_SEQ_ID        int64
DEST_CITY_MARKET_ID        int64
DEST                      object
DEST_CITY_NAME            object
DEST_WAC                   int64
CRS_DEP_TIME               int64
DEP_TIME                 float64
DEP_DELAY                float64
CRS_ARR_TIME               int64
ARR_TIME                 float64
ARR_DELAY                float64
CANCELLED                float64
CANCELLATION_CODE         object
DIVERTED                 float64
CRS_ELAPSED_TIME         float64
ACTUAL_ELAPSED_TIME      float64
AIR_TIME  

Creating the Fact Table and transforming the fact table

In [6]:
fact_table = flights_db[["CANCELLATION_CODE", 
                      "FL_DATE", 
                      "OP_CARRIER_AIRLINE_ID", 
                      "TAIL_NUM", 
                      "ORIGIN_AIRPORT_ID", 
                      "DEST_AIRPORT_ID", 
                      "DISTANCE",
                      "CRS_ELAPSED_TIME", 
                      "ACTUAL_ELAPSED_TIME",
                      "AIR_TIME",
                      "CARRIER_DELAY", 
                      "WEATHER_DELAY", 
                      "NAS_DELAY", 
                      "SECURITY_DELAY", 
                      "LATE_AIRCRAFT_DELAY"
                      ]]

fact_table["IS_CANCELLED"] = np.where(fact_table["CANCELLATION_CODE"].isna(), 1, 0)

C:\Users\jenso\AppData\Local\Temp\ipykernel_16256\865295849.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fact_table["IS_CANCELLED"] = np.where(fact_table["CANCELLATION_CODE"].isna(), 1, 0)


In [7]:
fact_table.isnull().sum()

CANCELLATION_CODE        504646
FL_DATE                       0
OP_CARRIER_AIRLINE_ID         0
TAIL_NUM                   8473
ORIGIN_AIRPORT_ID             0
DEST_AIRPORT_ID               0
DISTANCE                      0
CRS_ELAPSED_TIME              0
ACTUAL_ELAPSED_TIME       34373
AIR_TIME                  34373
CARRIER_DELAY            440850
WEATHER_DELAY            440850
NAS_DELAY                440850
SECURITY_DELAY           440850
LATE_AIRCRAFT_DELAY      440850
IS_CANCELLED                  0
dtype: int64

In [8]:
#Handling null value
fact_table["CARRIER_DELAY"] = fact_table["CARRIER_DELAY"].fillna(0)
fact_table["WEATHER_DELAY"] = fact_table["WEATHER_DELAY"].fillna(0)
fact_table["SECURITY_DELAY"] = fact_table["SECURITY_DELAY"].fillna(0)
fact_table["NAS_DELAY"] = fact_table["NAS_DELAY"].fillna(0)
fact_table["LATE_AIRCRAFT_DELAY"] = fact_table["LATE_AIRCRAFT_DELAY"].fillna(0)

fact_table["TAIL_NUM"] = fact_table["TAIL_NUM"].fillna("unknown")
fact_table["CANCELLATION_CODE"] = fact_table["CANCELLATION_CODE"].fillna("E")

fact_table["AIR_TIME"] = fact_table["AIR_TIME"].fillna(0)
fact_table["ACTUAL_ELAPSED_TIME"] = fact_table["ACTUAL_ELAPSED_TIME"].fillna(0)
fact_table["CRS_ELAPSED_TIME"] = fact_table["CRS_ELAPSED_TIME"].fillna(0)


C:\Users\jenso\AppData\Local\Temp\ipykernel_16256\2917149582.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fact_table["CARRIER_DELAY"] = fact_table["CARRIER_DELAY"].fillna(0)
C:\Users\jenso\AppData\Local\Temp\ipykernel_16256\2917149582.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fact_table["WEATHER_DELAY"] = fact_table["WEATHER_DELAY"].fillna(0)
C:\Users\jenso\AppData\Local\Temp\ipykernel_16256\2917149582.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a Da

In [9]:
fact_table.isnull().sum()

CANCELLATION_CODE        0
FL_DATE                  0
OP_CARRIER_AIRLINE_ID    0
TAIL_NUM                 0
ORIGIN_AIRPORT_ID        0
DEST_AIRPORT_ID          0
DISTANCE                 0
CRS_ELAPSED_TIME         0
ACTUAL_ELAPSED_TIME      0
AIR_TIME                 0
CARRIER_DELAY            0
WEATHER_DELAY            0
NAS_DELAY                0
SECURITY_DELAY           0
LATE_AIRCRAFT_DELAY      0
IS_CANCELLED             0
dtype: int64

In [10]:
fact_table["Fact Table ID"] = fact_table.index
fact_table = fact_table[[
                        "Fact Table ID",
                        "CANCELLATION_CODE", 
                        "FL_DATE", 
                        "OP_CARRIER_AIRLINE_ID", 
                        "TAIL_NUM", 
                        "ORIGIN_AIRPORT_ID", 
                        "DEST_AIRPORT_ID", 
                        "DISTANCE",
                        "CRS_ELAPSED_TIME", 
                        "ACTUAL_ELAPSED_TIME",
                        "AIR_TIME",
                        "CARRIER_DELAY", 
                        "WEATHER_DELAY", 
                        "NAS_DELAY", 
                        "SECURITY_DELAY", 
                        "LATE_AIRCRAFT_DELAY",
                        "IS_CANCELLED"]]
fact_table

C:\Users\jenso\AppData\Local\Temp\ipykernel_16256\1243326853.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fact_table["Fact Table ID"] = fact_table.index


,Fact Table ID,CANCELLATION_CODE,FL_DATE,OP_CARRIER_AIRLINE_ID,TAIL_NUM,ORIGIN_AIRPORT_ID,DEST_AIRPORT_ID,DISTANCE,CRS_ELAPSED_TIME,ACTUAL_ELAPSED_TIME,AIR_TIME,CARRIER_DELAY,WEATHER_DELAY,NAS_DELAY,SECURITY_DELAY,LATE_AIRCRAFT_DELAY,IS_CANCELLED
0,0,B,1/1/2022 12:00:00 AM,19393,unknown,13796,11292,957.0,145.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
1,1,B,1/1/2022 12:00:00 AM,19393,unknown,10693,13204,616.0,110.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
2,2,B,1/1/2022 12:00:00 AM,19393,unknown,11292,13232,895.0,140.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
3,3,B,1/1/2022 12:00:00 AM,19393,unknown,12953,13232,725.0,150.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
4,4,B,1/1/2022 12:00:00 AM,19393,unknown,13198,15304,1048.0,150.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
537897,537897,E,1/31/2022 12:00:00 AM,20452,N879RW,14122,12953,335.0,90.0,75.0,55.0,0.0,0.0,0.0,0.0,0.0,1
537898,537898,E,1/31/2022 12:00:00 AM,20452,N979RP,11618,11624,1196.0,215.0,202.0,169.0,0.0,0.0,0.0,0.0,0.0,1
537899,537899,E,1/31/2022 12:00:00 AM,20452,N979RP,11624,11618,1196.0,193.0,194.0,169.0,28.0,0.0,1.0,0.0,0.0,1
537900,537900,E,1/31/2022 12:00:00 AM,20452,N979RP,14685,11618,708.0,135.0,111.0,97.0,0.0,0.0,0.0,0.0,0.0,1


In [11]:
# Function to change the first character
def change_first_char(text):
    if text.startswith('N'):
        return text[1:]
    else:
        return text
    
fact_table['TAIL_NUM'] = fact_table['TAIL_NUM'].apply(change_first_char)

In [12]:
fact_table

,Fact Table ID,CANCELLATION_CODE,FL_DATE,OP_CARRIER_AIRLINE_ID,TAIL_NUM,ORIGIN_AIRPORT_ID,DEST_AIRPORT_ID,DISTANCE,CRS_ELAPSED_TIME,ACTUAL_ELAPSED_TIME,AIR_TIME,CARRIER_DELAY,WEATHER_DELAY,NAS_DELAY,SECURITY_DELAY,LATE_AIRCRAFT_DELAY,IS_CANCELLED
0,0,B,1/1/2022 12:00:00 AM,19393,unknown,13796,11292,957.0,145.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
1,1,B,1/1/2022 12:00:00 AM,19393,unknown,10693,13204,616.0,110.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
2,2,B,1/1/2022 12:00:00 AM,19393,unknown,11292,13232,895.0,140.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
3,3,B,1/1/2022 12:00:00 AM,19393,unknown,12953,13232,725.0,150.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
4,4,B,1/1/2022 12:00:00 AM,19393,unknown,13198,15304,1048.0,150.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
537897,537897,E,1/31/2022 12:00:00 AM,20452,879RW,14122,12953,335.0,90.0,75.0,55.0,0.0,0.0,0.0,0.0,0.0,1
537898,537898,E,1/31/2022 12:00:00 AM,20452,979RP,11618,11624,1196.0,215.0,202.0,169.0,0.0,0.0,0.0,0.0,0.0,1
537899,537899,E,1/31/2022 12:00:00 AM,20452,979RP,11624,11618,1196.0,193.0,194.0,169.0,28.0,0.0,1.0,0.0,0.0,1
537900,537900,E,1/31/2022 12:00:00 AM,20452,979RP,14685,11618,708.0,135.0,111.0,97.0,0.0,0.0,0.0,0.0,0.0,1


Creating Cancellation Dimension

In [13]:
cancellation = cancellation_look_up[["Code", "Description"]]


In [14]:

cancellation["Cancellation ID"] = cancellation.index

In [15]:
cancellation = cancellation[["Cancellation ID","Code", "Description"]]

In [16]:
cancellation

,Cancellation ID,Code,Description
0,0,A,Carrier
1,1,B,Weather
2,2,C,National Air System
3,3,D,Security
4,4,E,No Cancellation


Creating Aircraft Dimensions

In [17]:
aircrafts = aircrafts_lookup[["N-NUMBER", "NAME"]]

In [18]:
aircrafts.isnull().sum()

N-NUMBER    0
NAME        1
dtype: int64

In [19]:
aircrafts["Aircraft ID"] = aircrafts.index
aircrafts.rename(columns={'N-NUMBER':'TAIL_NUM'}, inplace=True)
aircrafts =  aircrafts[["Aircraft ID", "TAIL_NUM", "NAME"]]
aircrafts["NAME"] = aircrafts["NAME"].fillna("unknown")
aircrafts

C:\Users\jenso\AppData\Local\Temp\ipykernel_16256\889505696.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  aircrafts["Aircraft ID"] = aircrafts.index
C:\Users\jenso\AppData\Local\Temp\ipykernel_16256\889505696.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  aircrafts.rename(columns={'N-NUMBER':'TAIL_NUM'}, inplace=True)


,Aircraft ID,TAIL_NUM,NAME
0,0,1,TENAX AEROSPACE LLC ...
1,1,100,BENE MARY D ...
2,2,10001,STOOS ROBERT A ...
3,3,10004,ETOS AIR LLC ...
4,4,10006,COUTCHES ROBERT HERCULES DBA ...
...,...,...,...
293461,293461,9ZS,COLLINS BRIAN D ...
293462,293462,9ZT,VALIAIR LC TRUSTEE ...
293463,293463,9ZU,FOWLER RONALD W ...
293464,293464,9ZX,HOOVER MICHAEL B ...


Creating Flight Date Dimension

In [20]:
flight_date = flights_db[["FL_DATE", "YEAR", "MONTH", "DAY_OF_MONTH"]].drop_duplicates()

In [21]:
flight_date.isnull().sum()

FL_DATE         0
YEAR            0
MONTH           0
DAY_OF_MONTH    0
dtype: int64

In [22]:
flight_date.dtypes

FL_DATE         object
YEAR             int64
MONTH            int64
DAY_OF_MONTH     int64
dtype: object

In [23]:
flight_date = flight_date.reset_index(drop=True)
flight_date["Flight Date ID"] = flight_date.index
flight_date = flight_date[["Flight Date ID", "FL_DATE", "YEAR", "MONTH", "DAY_OF_MONTH"]]

In [24]:
flight_date

,Flight Date ID,FL_DATE,YEAR,MONTH,DAY_OF_MONTH
0,0,1/1/2022 12:00:00 AM,2022,1,1
1,1,1/2/2022 12:00:00 AM,2022,1,2
2,2,1/3/2022 12:00:00 AM,2022,1,3
3,3,1/4/2022 12:00:00 AM,2022,1,4
4,4,1/5/2022 12:00:00 AM,2022,1,5
5,5,1/6/2022 12:00:00 AM,2022,1,6
6,6,1/7/2022 12:00:00 AM,2022,1,7
7,7,1/8/2022 12:00:00 AM,2022,1,8
8,8,1/9/2022 12:00:00 AM,2022,1,9
9,9,1/10/2022 12:00:00 AM,2022,1,10


Creating Airline Dimension

In [25]:
airline = flights_db["OP_CARRIER_AIRLINE_ID"].drop_duplicates()

In [26]:
airline = pd.merge(airline, airline_look_up, left_on="OP_CARRIER_AIRLINE_ID", right_on="Code")

In [27]:
airline = airline[["OP_CARRIER_AIRLINE_ID", "Description"]]

In [28]:
airline.dtypes

OP_CARRIER_AIRLINE_ID     int64
Description              object
dtype: object

In [29]:
airline["Description"] = airline["Description"].astype(str)

In [30]:
airline["Airline ID"] = airline.index
airline = airline[["Airline ID", "OP_CARRIER_AIRLINE_ID", "Description"]]

In [31]:
airline

,Airline ID,OP_CARRIER_AIRLINE_ID,Description
0,0,19393,Southwest Airlines Co.: WN
1,1,19687,Horizon Air: QX
2,2,19690,Hawaiian Airlines Inc.: HA
3,3,19790,Delta Air Lines Inc.: DL
4,4,19805,American Airlines Inc.: AA
5,5,19930,Alaska Airlines Inc.: AS
6,6,19977,United Air Lines Inc.: UA
7,7,20304,SkyWest Airlines Inc.: OO
8,8,20363,Endeavor Air Inc.: 9E
9,9,20368,Allegiant Air: G4


Creating Arrival Airport Dimension

In [32]:
origin_airport = flights_db[["ORIGIN_AIRPORT_ID","ORIGIN_CITY_NAME","ORIGIN_WAC"]].drop_duplicates()

In [33]:
origin_airport = pd.merge(origin_airport, airport_look_up, left_on="ORIGIN_AIRPORT_ID",right_on="Code")

In [34]:
origin_airport = origin_airport[["ORIGIN_AIRPORT_ID","ORIGIN_CITY_NAME","ORIGIN_WAC", "Description"]]

In [35]:
origin_airport = pd.merge(origin_airport, country_look_up, left_on="ORIGIN_WAC",right_on="Code")

In [36]:
origin_airport = origin_airport[["ORIGIN_AIRPORT_ID","Description_y", "ORIGIN_CITY_NAME", "Description_x"]]

In [37]:
origin_airport.columns = ["ORIGIN_AIRPORT_ID", "State", "City", "Name"]

In [38]:
origin_airport.isnull().sum()

ORIGIN_AIRPORT_ID    0
State                0
City                 0
Name                 0
dtype: int64

In [39]:
origin_airport.dtypes

ORIGIN_AIRPORT_ID     int64
State                object
City                 object
Name                 object
dtype: object

In [40]:
origin_airport["City"] = origin_airport["City"].astype(str)
origin_airport["Name"] = origin_airport["Name"].astype(str)
origin_airport["State"] = origin_airport["State"].astype(str)

In [41]:
origin_airport["Origin Airport Surrogate Key"] = origin_airport.index
origin_airport = origin_airport[["Origin Airport Surrogate Key", "ORIGIN_AIRPORT_ID", "State", "City", "Name"]]

In [42]:
origin_airport

,Origin Airport Surrogate Key,ORIGIN_AIRPORT_ID,State,City,Name
0,0,13796,California,"Oakland, CA","Oakland, CA: Metro Oakland International"
1,1,10693,Tennessee,"Nashville, TN","Nashville, TN: Nashville International"
2,2,11292,Colorado,"Denver, CO","Denver, CO: Denver International"
3,3,12953,New York,"New York, NY","New York, NY: LaGuardia"
4,4,13198,Missouri,"Kansas City, MO","Kansas City, MO: Kansas City International"
...,...,...,...,...,...
358,358,12320,Delaware,"Wilmington, DE","Wilmington, DE: New Castle"
359,359,14098,Virginia,"Newport News/Williamsburg, VA","Newport News/Williamsburg, VA: Newport News/Wi..."
360,360,13139,Virginia,"Lynchburg, VA","Lynchburg, VA: Lynchburg Regional/Preston Glen..."
361,361,14222,U.S. Pacific Trust Territories and Possessions,"Pago Pago, TT","Pago Pago, TT: Pago Pago International"


Creating the Destination Airport

In [43]:
destination_airport = flights_db[["DEST_AIRPORT_ID","DEST_CITY_NAME","DEST_WAC"]].drop_duplicates()

In [44]:
destination_airport = pd.merge(destination_airport, airport_look_up, left_on="DEST_AIRPORT_ID",right_on="Code")

In [45]:
destination_airport = destination_airport[["DEST_AIRPORT_ID","DEST_CITY_NAME","DEST_WAC", "Description"]]

In [46]:
destination_airport = pd.merge(destination_airport, country_look_up, left_on="DEST_WAC",right_on="Code")

In [47]:
destination_airport = destination_airport[["DEST_AIRPORT_ID","Description_y", "DEST_CITY_NAME", "Description_x"]]

In [48]:
destination_airport.columns = ["DEST_AIRPORT_ID", "State", "City", "Name"]

In [49]:
destination_airport.isnull().sum()

DEST_AIRPORT_ID    0
State              0
City               0
Name               0
dtype: int64

In [50]:
destination_airport.dtypes

DEST_AIRPORT_ID     int64
State              object
City               object
Name               object
dtype: object

In [51]:
destination_airport["City"] = destination_airport["City"].astype(str)
destination_airport["Name"] = destination_airport["Name"].astype(str)
destination_airport["State"] = destination_airport["State"].astype(str)

In [52]:
destination_airport["Destination Airport Surrogate Key"] = destination_airport.index
destination_airport = destination_airport[["Destination Airport Surrogate Key", "DEST_AIRPORT_ID", "State", "City", "Name"]]

In [53]:
destination_airport

,Destination Airport Surrogate Key,DEST_AIRPORT_ID,State,City,Name
0,0,11292,Colorado,"Denver, CO","Denver, CO: Denver International"
1,1,13204,Florida,"Orlando, FL","Orlando, FL: Orlando International"
2,2,13232,Illinois,"Chicago, IL","Chicago, IL: Chicago Midway International"
3,3,15304,Florida,"Tampa, FL","Tampa, FL: Tampa International"
4,4,10140,New Mexico,"Albuquerque, NM","Albuquerque, NM: Albuquerque International Sun..."
...,...,...,...,...,...
358,358,12320,Delaware,"Wilmington, DE","Wilmington, DE: New Castle"
359,359,14098,Virginia,"Newport News/Williamsburg, VA","Newport News/Williamsburg, VA: Newport News/Wi..."
360,360,13139,Virginia,"Lynchburg, VA","Lynchburg, VA: Lynchburg Regional/Preston Glen..."
361,361,14222,U.S. Pacific Trust Territories and Possessions,"Pago Pago, TT","Pago Pago, TT: Pago Pago International"


Placing the surrogate key in fact table

In [54]:
fact_table["Cancellation Surrogate Key"] = pd.merge(fact_table, cancellation, left_on="CANCELLATION_CODE", right_on="Code")["Cancellation ID"]



In [55]:
fact_table["Airline Surrogate Key"] = pd.merge(fact_table, airline, left_on="OP_CARRIER_AIRLINE_ID", right_on="OP_CARRIER_AIRLINE_ID")["Airline ID"]


In [56]:
fact_table["Aircraft Surrogate Key"] = fact_table.merge(aircrafts, how='left', on='TAIL_NUM')["Aircraft ID"]

In [57]:
fact_table["Origin Airport Surrogate Key"] = pd.merge(fact_table, origin_airport, left_on="ORIGIN_AIRPORT_ID", right_on="ORIGIN_AIRPORT_ID")["Origin Airport Surrogate Key"]


In [58]:
fact_table["Destination Airport Surrogate Key"] = pd.merge(fact_table, destination_airport, left_on="DEST_AIRPORT_ID", right_on="DEST_AIRPORT_ID")["Destination Airport Surrogate Key"]


In [59]:
fact_table["Flight Date Surrogate Key"] = pd.merge(fact_table, flight_date, how="left", left_on="FL_DATE" , right_on="FL_DATE")["Flight Date ID"]

In [60]:
fact_table

,Fact Table ID,CANCELLATION_CODE,FL_DATE,OP_CARRIER_AIRLINE_ID,TAIL_NUM,ORIGIN_AIRPORT_ID,DEST_AIRPORT_ID,DISTANCE,CRS_ELAPSED_TIME,ACTUAL_ELAPSED_TIME,...,NAS_DELAY,SECURITY_DELAY,LATE_AIRCRAFT_DELAY,IS_CANCELLED,Cancellation Surrogate Key,Airline Surrogate Key,Aircraft Surrogate Key,Origin Airport Surrogate Key,Destination Airport Surrogate Key,Flight Date Surrogate Key
0,0,B,1/1/2022 12:00:00 AM,19393,unknown,13796,11292,957.0,145.0,0.0,...,0.0,0.0,0.0,0,1,0,293465.0,0,0,0
1,1,B,1/1/2022 12:00:00 AM,19393,unknown,10693,13204,616.0,110.0,0.0,...,0.0,0.0,0.0,0,1,0,293465.0,1,1,0
2,2,B,1/1/2022 12:00:00 AM,19393,unknown,11292,13232,895.0,140.0,0.0,...,0.0,0.0,0.0,0,1,0,293465.0,2,2,0
3,3,B,1/1/2022 12:00:00 AM,19393,unknown,12953,13232,725.0,150.0,0.0,...,0.0,0.0,0.0,0,1,0,293465.0,3,2,0
4,4,B,1/1/2022 12:00:00 AM,19393,unknown,13198,15304,1048.0,150.0,0.0,...,0.0,0.0,0.0,0,1,0,293465.0,4,3,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
537897,537897,E,1/31/2022 12:00:00 AM,20452,879RW,14122,12953,335.0,90.0,75.0,...,0.0,0.0,0.0,1,4,16,255764.0,51,40,30
537898,537898,E,1/31/2022 12:00:00 AM,20452,979RP,11618,11624,1196.0,215.0,202.0,...,0.0,0.0,0.0,1,4,16,286902.0,130,131,30
537899,537899,E,1/31/2022 12:00:00 AM,20452,979RP,11624,11618,1196.0,193.0,194.0,...,1.0,0.0,0.0,1,4,16,286902.0,131,130,30
537900,537900,E,1/31/2022 12:00:00 AM,20452,979RP,14685,11618,708.0,135.0,111.0,...,0.0,0.0,0.0,1,4,16,286902.0,87,130,30


In [61]:
fact_table = fact_table[[
                        "Fact Table ID",
                        "Cancellation Surrogate Key", 
                        "Flight Date Surrogate Key", 
                        "Airline Surrogate Key", 
                        "Aircraft Surrogate Key", 
                        "Origin Airport Surrogate Key", 
                        "Destination Airport Surrogate Key", 
                        "DISTANCE",
                        "CRS_ELAPSED_TIME", 
                        "ACTUAL_ELAPSED_TIME",
                        "AIR_TIME",
                        "CARRIER_DELAY", 
                        "WEATHER_DELAY", 
                        "NAS_DELAY", 
                        "SECURITY_DELAY", 
                        "LATE_AIRCRAFT_DELAY", 
                        "IS_CANCELLED"]]

In [62]:
fact_table.isnull().sum()

Fact Table ID                           0
Cancellation Surrogate Key              0
Flight Date Surrogate Key               0
Airline Surrogate Key                   0
Aircraft Surrogate Key               6194
Origin Airport Surrogate Key            0
Destination Airport Surrogate Key       0
DISTANCE                                0
CRS_ELAPSED_TIME                        0
ACTUAL_ELAPSED_TIME                     0
AIR_TIME                                0
CARRIER_DELAY                           0
WEATHER_DELAY                           0
NAS_DELAY                               0
SECURITY_DELAY                          0
LATE_AIRCRAFT_DELAY                     0
IS_CANCELLED                            0
dtype: int64

In [63]:
fact_table["Aircraft Surrogate Key"] =fact_table["Aircraft Surrogate Key"].fillna(293465)

C:\Users\jenso\AppData\Local\Temp\ipykernel_16256\497463755.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fact_table["Aircraft Surrogate Key"] =fact_table["Aircraft Surrogate Key"].fillna(293465)


Displaying Dimension and Fact Table

In [64]:
aircrafts
aircrafts.to_csv("Aircraft.csv", index=None)

In [65]:
flight_date
flight_date.to_csv("Flight Date.csv", index=None)

In [66]:
airline
airline.to_csv("Airline.csv", index=None)

In [67]:
origin_airport
origin_airport.to_csv("Origin Airport.csv", index=None)

In [68]:
destination_airport
destination_airport.to_csv("Destination Airport.csv", index=None)

In [69]:
cancellation
cancellation.to_csv("Cancellation.csv", index=None)

In [70]:
fact_table = fact_table.astype(int)

In [71]:
fact_table


fact_table.to_csv("Fact Table.csv", index=None)